# M5 F1 - Promos

* Flag one oe two weeks price changes
* Scaled prices
* Price pct change

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

In [2]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### Load Datasets
All three datasets needed because we need to calculate sales in USD.

In [3]:
# Items with the most often promos:
ids = ['FOODS_3_578',
'FOODS_3_610',
'FOODS_3_615',
'FOODS_3_348',
'FOODS_3_785']

In [4]:
# Load grid with gaps and imputed sales:
cols = ['id', 'item_id', 'store_id', 'level', 'wm_yr_wk_x', 'sell_price', 'sales']

file_pass = '/kaggle/input/m5-s2-holidays/grid_s2.pkl'
grid_df = pd.read_pickle(file_pass)[cols]
grid_df.rename(columns={"wm_yr_wk_x": "wm_yr_wk"}, inplace=True)

In [5]:
# Create week aggregations:
sell_prices = grid_df.groupby(['id','wm_yr_wk']).mean().fillna(0)
sell_prices.reset_index(inplace=True)

# Sales features:

In [6]:
# Shifted prices:
sell_prices['price_after2'] = sell_prices.groupby(['id'])['sell_price'].shift(-2)
sell_prices['price_after'] = sell_prices.groupby(['id'])['sell_price'].shift(-1)
sell_prices['price_before'] = sell_prices.groupby(['id'])['sell_price'].shift(1)
sell_prices['price_before2'] = sell_prices.groupby(['id'])['sell_price'].shift(2)

# One week sale is when current price is not equal to the previous 
# but previous is equal to future:
sell_prices['x1_week_sale'] = (
        sell_prices.sell_price != sell_prices.price_before
    ) & (
        sell_prices.price_before == sell_prices.price_after
    ) & (sell_prices.price_before > 0)


# Two weeks sale is when current price is not equal to the previous, equal to next 
# and previous is equal to the one after next:
sell_prices['x2_week_sale'] = (
        sell_prices.sell_price != sell_prices.price_before
    ) & (
        sell_prices.sell_price == sell_prices.price_after
    ) & (
        sell_prices.price_before == sell_prices.price_after2
    ) & (sell_prices.price_before > 0)

sell_prices['x2_week_sale2'] = (
        sell_prices.sell_price != sell_prices.price_before2
    ) & (
        sell_prices.sell_price == sell_prices.price_before
    ) & (
        sell_prices.price_before2 == sell_prices.price_after
    ) & (sell_prices.price_before2 > 0)

# Promo weeks flag:
sell_prices['promo_weeks_flag'] = sell_prices.x1_week_sale|sell_prices.x2_week_sale|sell_prices.x2_week_sale2
sell_prices['promo_weeks_revenue'] = sell_prices['promo_weeks_flag']*sell_prices['sales']*sell_prices['sell_price']

In [7]:
# only useful columns:
cols = ['id', 'wm_yr_wk', 'promo_weeks_flag','sell_price','promo_weeks_revenue']
sell_prices = sell_prices[cols]

# Rollup

In [8]:
# 1. Load roll_up dataframe
# Function to do quick rollups:
def rollup(v):
    '''
    v - np.array of size (30490 rows, n day columns)
    v_rolledup - array of size (n, 42840)
    '''
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T

# Load CSR matrix for quick Rollups:
## Define folder pass here:
file_pass = '/kaggle/input/fast-clear-wrmsse-18ms/'# '/kaggle/input/fast-wrmsse-and-sw-frame/'

## Load matrix and save rollup index:
roll_mat_df = reduce_mem_usage(pd.read_pickle(file_pass+'roll_mat_df.pkl'))
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
del roll_mat_df

Mem. usage decreased to 1247.46 Mb (0.0% reduction)


In [9]:
# Grid_df and Csr_matrix indices should be sorted in the same way:
sorted_idx = roll_index.to_frame().loc[11].id.index

# Prepare for Rollup by making every row an ID:
sell_prices = sell_prices.pivot(index='id', columns='wm_yr_wk').astype(np.float32).reindex(sorted_idx)

# Save columns for future use:
week_cols = sell_prices.columns

# Rollup by multiplying by csr matrix:
roll_week_df = pd.DataFrame(rollup(sell_prices.values), index = roll_index, columns = week_cols)

# Return to long form:
roll_week_df = roll_week_df.T.unstack(level=1).T.reset_index()

# Reduce memory:
roll_week_df = reduce_mem_usage(roll_week_df)

del sell_prices, grid_df
gc.collect()

Mem. usage decreased to 215.80 Mb (47.2% reduction)


0

In [10]:
# Create price features:
roll_week_df['price_pct_change'] = roll_week_df.groupby(['id'])['sell_price'].pct_change()
roll_week_df['scaled_price'] = roll_week_df.groupby(['id'])['sell_price'].apply(lambda x: x/max(x))
roll_week_df['promo_weeks_revenue'] = roll_week_df.groupby(['id'])['promo_weeks_revenue'].apply(lambda x: np.log(x+1))

In [11]:
roll_week_df.to_pickle('m5_f1_promos.pkl')

In [12]:
roll_week_df.head()

,level,id,wm_yr_wk,promo_weeks_flag,sell_price,promo_weeks_revenue,price_pct_change,scaled_price
0,0,all,11101,0.0,44506.437500,0.000000,NaN,0.325461
1,0,all,11102,2.0,49834.167969,1.411133,0.119707,0.364420
2,0,all,11103,4.0,52459.410156,1.109375,0.052680,0.383618
3,0,all,11104,9.0,53765.269531,3.416016,0.024893,0.393167
4,0,all,11105,6.0,54776.277344,2.617188,0.018804,0.400560
